### Project Chicago: Transform ResStock PACKAGE Results to 15 SFD prototypes (Elevate Energy)
Created on: 01/07/2020 \
By: Lixi Liu (Lixi.Liu@nrel.gov)

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
print(f'Notebook path: {os.getcwd()}')

Notebook path: /Users/lliu2/Documents/GitHub/ResStock/files


### Download results online
* unprocessed upgrade results: S3/resbldg-datasets/chicagoeui
* processed result tables for plots: https://nrel.sharepoint.com/sites/ChicagoRetrofits/Shared%20Documents/Forms/AllItems.aspx?viewid=289cdd1a%2D97c9%2D4bcc%2D8416%2Dc19bf01c6302&id=%2Fsites%2FChicagoRetrofits%2FShared%20Documents%2FGeneral%2FUpgrade%20results

### Initialize
For modifying plots, can go directly to section 2.3 after initialization

In [2]:
# local path to downloaded results
iteration = '03_sfamh' # <----- options: 03
iter_path = f'cookcnty_packages_{iteration}'
result_dir = '/Users/lliu2/Documents/Chicago retrofits/ResStock results'
result_path = os.path.join(result_dir, iter_path)

## create folder for post-processed results:
if not os.path.exists(os.path.join(result_path, 'processed results')):
    os.mkdir(os.path.join(result_path, 'processed results'))
    
## create folder for exported baseline and upgrade results to csv (to share with Elevate):
combined_res_csv_path = os.path.join(result_path, 'processed results', 'raw combined csvs')
if not os.path.exists(combined_res_csv_path):
    os.mkdir(combined_res_csv_path)
    
## create folder for plots:
plot_path = os.path.join(result_path, 'processed results', 'plots')
if not os.path.exists(plot_path):
    os.mkdir(plot_path)

print(f'Results path: \n   {result_path}')


Results path: 
   /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_03_sfamh


### Functions

In [3]:
def get_per_unit_sim_output(df, ref):
    """
    ref (df): baseline df
    """
    cols = [x for x in df.columns if 
            x.endswith('_kwh') or
            x.endswith('_mbtu') or
            x.endswith('_therm') or
            x.endswith('_cost_usd') or
            x.endswith('_ft_2')
           ]
    res = ref.set_index('building_id').reindex(df['building_id']).reset_index()
    df.loc[:, cols] = df.loc[:, cols].replace([None,''],np.nan).divide(
        res['build_existing_model.units_represented'], axis=0)
    
    return df
print('func loaded: "get_per_unit_sim_output"')

def get_per_unit_sim_output_limited(df, ref):
    """
    To reduce computing time
    ref (df): baseline df
    """
    cols = ['simulation_output_report.total_site_natural_gas_therm',
            'simulation_output_report.total_site_electricity_kwh',
            'simulation_output_report.total_site_energy_mbtu',
            'simulation_output_report.upgrade_cost_usd'
           ]
    res = ref.set_index('building_id').reindex(df['building_id']).reset_index()
    df.loc[:, cols] = df.loc[:, cols].replace([None,''],np.nan).divide(
        res['build_existing_model.units_represented'], axis=0)
    
    return df
print('func loaded: "get_per_unit_sim_output_limited"')

def add_sqft_eui(df, ref):
    """
    ARG:
        ref (df): baseline df
    RETURN:
        df with added cols: 'sqft', 'gas_eui_thermpersqft','elec_eui_kwhpersqft','site_eui_kbtupersqft'
    """
    res = ref.set_index('building_id').reindex(df['building_id']).reset_index()
    df['sqft'] = res['simulation_output_report.floor_area_conditioned_ft_2']
    df['gas_eui_thermpersqft'] = df['simulation_output_report.total_site_natural_gas_therm'].divide(df['sqft']) # therm/sqft
    df['elec_eui_kwhpersqft'] = df['simulation_output_report.total_site_electricity_kwh'].divide(df['sqft']) # kwh/sqft
    df['site_eui_kbtupersqft'] = df['simulation_output_report.total_site_energy_mbtu'].divide(df['sqft'])*1000 # kbtu/sqft
    
    for col in ['sqft','gas_eui_thermpersqft','elec_eui_kwhpersqft','site_eui_kbtupersqft']:
        df.loc[df['simulation_output_report.applicable']==False, col] = np.nan
    
    return df
print('func loaded: "add_sqft_eui"')

def get_res_by_prototype(df, filter_by_df, row):
    """
    ARG:
        df (dataframe): df to slice on
        filter_by_df (dataframe): df used to do the slicing
    RETURN:
        filtered df (dataframe)
    """
    slice_by_df = filter_by_df.copy().set_index('building_id').reindex(df['building_id']).reset_index()
    res_group_i = df[slice_by_df['build_existing_model.geometry_stories'].isin(row['Stories'].split(',')) & \
        slice_by_df['build_existing_model.geometry_wall_type'].isin(row['WallType'].split(',')) & \
        slice_by_df['build_existing_model.vintage_acs'].isin(row['Vintage'].split(',')) & \
        slice_by_df['build_existing_model.geometry_building_type_recs'].isin(['Single-Family Detached'])]
    res_group_i = res_group_i[res_group_i['completed_status']=="Success"]
    
    return res_group_i
print('func loaded: "get_res_by_prototype"')

def load_upgrade(n, file_dir=result_path):
    """
    ARG:
        n (int, str): upgrade number
        file_dir (str): folder in which upgrade can be found, default to main result dir
    RETURN:
        df (dataframe) of upgrade n
    """
    df = pd.read_parquet(os.path.join(file_dir,'upgrades',
                                     f'upgrade={n}/results_up{n:02d}.parquet'))
    return df
print('func loaded: "load_upgrade"')



func loaded: "get_per_unit_sim_output"
func loaded: "get_per_unit_sim_output_limited"
func loaded: "add_sqft_eui"
func loaded: "get_res_by_prototype"
func loaded: "load_upgrade"


### 1. BASELINE results

In [4]:
save_a_copy_in_csv = False # <-----
res = pd.read_parquet(os.path.join(result_path,'baseline','results_up00.parquet'))

# (1) get sqft, gas/elec/site eui
res = add_sqft_eui(res, res)
res['build_existing_model.sample_weight'] = 2173432/40000

if save_a_copy_in_csv:
    res.to_csv(os.path.join(combined_res_csv_path,'results_baseline.csv'), index=False)
    
# (1) get sim output at the unit level (req for MF)
res = get_per_unit_sim_output(res, res)

# (2) get sqft, gas/elec/site eui - redo to get unit-level results
res = add_sqft_eui(res, res)

# check
jobs_missing = set(range(1,100)) - set(res.job_id.unique())
print(f'- {len(jobs_missing)} jobs missing: {jobs_missing}')
print(f'- {16777-len(res)} buildings ({((16777-len(res))/16777*100):.2f}%)')

res

- 0 jobs missing: set()
- 14313 buildings (85.31%)


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,build_existing_model.ahs_region,build_existing_model.applicable,...,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,sqft,gas_eui_thermpersqft,elec_eui_kwhpersqft,site_eui_kbtupersqft
0,1,4,2021-05-24 09:32:54,2021-05-24 09:36:44,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,16.909091,18.272727,336.390504,319.082970,2514.0,395.017450,1202.0,0.445445,4.095515,58.508320
1,2,43,2021-05-24 09:30:30,2021-05-24 09:33:46,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,15.090909,16.363636,53.244335,67.353566,2155.0,74.409253,866.0,0.873858,7.004861,111.266492
2,3,6,2021-05-24 09:46:18,2021-05-24 09:50:02,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,16.000000,17.363636,141.767218,90.609550,4744.0,170.939906,1202.0,0.447239,3.904510,58.036003
3,4,85,2021-05-24 10:38:05,2021-05-24 10:41:27,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,14.454545,14.363636,247.485970,335.623777,544.0,421.112130,1202.0,0.000000,11.554596,39.425885
4,5,25,2021-05-24 08:54:48,2021-05-24 08:58:53,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,13.272727,13.909091,257.171182,278.261169,6378.0,376.538900,1675.0,0.288625,5.129941,46.359645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2460,51,2021-05-24 09:06:45,2021-05-24 09:08:02,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,16.727273,18.272727,6.247695,8.579762,8561.0,9.477889,1220.0,1.287834,6.335857,150.371562
2460,2461,51,2021-05-24 09:48:06,2021-05-24 09:51:51,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,15.727273,18.818182,114.457268,84.159643,3955.0,132.491715,1675.0,0.515991,3.814951,64.603972
2461,2462,21,2021-05-24 09:06:30,2021-05-24 09:10:23,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,14.181818,16.363636,47.371967,25.414189,4456.0,52.041861,1202.0,0.876230,4.198847,101.929126
2462,2463,32,2021-05-24 09:49:23,2021-05-24 09:53:21,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,14.727273,17.272727,103.564205,79.489322,5005.0,117.415387,1675.0,0.678793,4.000522,81.513476


In [5]:
# assign heating/cooling
cooling_col = 'build_existing_model.hvac_cooling_type'
heating_col = 'build_existing_model.hvac_heating_type_and_fuel'
print('selected')

# if using a national run
if iteration == '_national_2018':
    res = res[res['build_existing_model.ahs_region']=='CBSA Chicago-Naperville-Elgin, IL-IN-WI'].reset_index(drop=True)
    res
    

selected


### 1.1. Check housing charateristics distributions in BASELINE

In [6]:
proto_chars = ['build_existing_model.geometry_stories',
              'build_existing_model.geometry_wall_type',
              'build_existing_model.vintage_acs',
              cooling_col,
              heating_col,
              'build_existing_model.geometry_floor_area'
             ]

Ns = len(res.query('completed_status=="Success"')); N = len(res)
print(f'>>> ResStock - {iter_path} - BASELINE result summary:\n')
print(f'  * {Ns} / {N} samples ran successfully, {N-Ns} failed, efficacy: {Ns/N:.1%} \n')

print('>>> Housing characteristics splits:\n')

Res_char = []
for i, char in enumerate(proto_chars,1):
    Nchar = res.groupby(char)['building_id'].count()
    Nchar = Nchar/Ns
    Nchar['N_failed'] = len(res[res[char].isnull()])
    print(f'  * [{i}] {Nchar}\n')
    
    ## append for export
    Nchar = Nchar.rename('fraction').to_frame()
    Nchar['housing_char'] = Nchar.index.name
    Res_char.append(Nchar)
    
Res_char = pd.concat(Res_char, axis=0)
Res_char.index.name = 'sub_char'
Res_char = Res_char.reset_index()
Res_char = Res_char[['housing_char','sub_char','fraction']]

Res_char.to_csv(os.path.join(result_path, 'baseline', 'baseline_housing_char_breakdown.csv'), index=False)
print(f'>>> file saved to:\n  {os.path.join(result_path, "baseline")}')

>>> ResStock - cookcnty_packages_03_sfamh - BASELINE result summary:

  * 2464 / 2464 samples ran successfully, 0 failed, efficacy: 100.0% 

>>> Housing characteristics splits:

  * [1] build_existing_model.geometry_stories
1           0.489854
2           0.455763
3           0.054383
N_failed    0.000000
Name: building_id, dtype: float64

  * [2] build_existing_model.geometry_wall_type
Masonry     0.505682
WoodStud    0.494318
N_failed    0.000000
Name: building_id, dtype: float64

  * [3] build_existing_model.vintage_acs
1940-59     0.136769
1960-79     0.242289
1980-99     0.353896
2000-09     0.164367
2010s       0.014610
<1940       0.088068
N_failed    0.000000
Name: building_id, dtype: float64

  * [4] build_existing_model.hvac_cooling_type
Central AC    0.807630
Heat Pump     0.025162
None          0.026380
Room AC       0.140828
N_failed      0.000000
Name: building_id, dtype: float64

  * [5] build_existing_model.hvac_heating_type_and_fuel
Electricity ASHP                   

### 1.2. Add additional metrics to Baseline

In [7]:
## EE prototype tags
res['vintage_ee'] = '3: post-1978'
res.loc[res['build_existing_model.vintage_acs'].isin(['1940-59','1960-79']),'vintage_ee'] = '2: 1942-1978'
res.loc[res['build_existing_model.vintage_acs']=='<1940','vintage_ee'] = '1: pre-1942'

res['stories'] = '2: 2+ stories'
res.loc[res['build_existing_model.geometry_stories']=='1','stories'] = '1: <2 stories'

print('new cols added to "res"')
res

new cols added to "res"


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,build_existing_model.ahs_region,build_existing_model.applicable,...,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,sqft,gas_eui_thermpersqft,elec_eui_kwhpersqft,site_eui_kbtupersqft,vintage_ee,stories
0,1,4,2021-05-24 09:32:54,2021-05-24 09:36:44,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,336.390504,319.082970,2514.0,395.017450,1202.0,0.445445,4.095515,58.508320,3: post-1978,2: 2+ stories
1,2,43,2021-05-24 09:30:30,2021-05-24 09:33:46,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,53.244335,67.353566,2155.0,74.409253,866.0,0.873858,7.004861,111.266492,2: 1942-1978,1: <2 stories
2,3,6,2021-05-24 09:46:18,2021-05-24 09:50:02,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,141.767218,90.609550,4744.0,170.939906,1202.0,0.447239,3.904510,58.036003,3: post-1978,2: 2+ stories
3,4,85,2021-05-24 10:38:05,2021-05-24 10:41:27,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,247.485970,335.623777,544.0,421.112130,1202.0,0.000000,11.554596,39.425885,3: post-1978,1: <2 stories
4,5,25,2021-05-24 08:54:48,2021-05-24 08:58:53,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,257.171182,278.261169,6378.0,376.538900,1675.0,0.288625,5.129941,46.359645,3: post-1978,2: 2+ stories
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2460,51,2021-05-24 09:06:45,2021-05-24 09:08:02,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,6.247695,8.579762,8561.0,9.477889,1220.0,1.287834,6.335857,150.371562,1: pre-1942,1: <2 stories
2460,2461,51,2021-05-24 09:48:06,2021-05-24 09:51:51,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,114.457268,84.159643,3955.0,132.491715,1675.0,0.515991,3.814951,64.603972,2: 1942-1978,2: 2+ stories
2461,2462,21,2021-05-24 09:06:30,2021-05-24 09:10:23,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,47.371967,25.414189,4456.0,52.041861,1202.0,0.876230,4.198847,101.929126,3: post-1978,2: 2+ stories
2462,2463,32,2021-05-24 09:49:23,2021-05-24 09:53:21,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,103.564205,79.489322,5005.0,117.415387,1675.0,0.678793,4.000522,81.513476,3: post-1978,2: 2+ stories


### 1.3. Get BASELINE results by building prototypes from Elevate Energy
#### Note: cannot add prototype tags directly to ResStock results as prototypes overlap in chars

In [8]:
## Load prototype csv
groups = pd.read_csv(os.path.join(result_dir,'Groups.csv'))
print('prototypes loaded')


prototypes loaded


In [9]:
### (1) get count & median consumption
Metric_map1 = {'count': 'completed_status',
           'median gas': 'simulation_output_report.total_site_natural_gas_therm',
           'median elec': 'simulation_output_report.total_site_electricity_kwh'}

for i, row in groups.iterrows():
    res_group_i = get_res_by_prototype(res, res, row) # <----
    
    for metric, res_var in Metric_map1.items():
        if metric == 'count':
            groups.loc[i, 'count'] = len(res_group_i)
        else:
            groups.loc[i, metric] = res_group_i[res_var].median()

groups['Gas Diff-med(%)'] = ((groups['Non-normalized gas usage'] - groups['median gas']) / groups['Non-normalized gas usage']) * 100
groups['Elec Diff-med(%)'] = ((groups['Non-normalized elec. usage'] - groups['median elec']) / groups['Non-normalized elec. usage']) * 100
print('>> (1) median consumption computed')


>> (1) median consumption computed


In [10]:
res2 = []

for i, row in groups.iterrows():
    if row['HousingGroupName'] not in \
        ['Masonry All Years Split Level','Frame Post-1978 Split Level','Frame Pre-1942 Split Level']:
        res_group_i = get_res_by_prototype(res, res, row) # <----
        res_group_i['prototype'] = row['HousingGroupName']
        res2.append(res_group_i)

res2 = pd.concat(res2, axis=0).sort_values(by=['building_id'])
display(res2)

res2 = res2.set_index('building_id').reindex(res['building_id']).reset_index()
res.loc[res2['prototype'].isnull(),:]

,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,build_existing_model.ahs_region,build_existing_model.applicable,...,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,sqft,gas_eui_thermpersqft,elec_eui_kwhpersqft,site_eui_kbtupersqft,vintage_ee,stories,prototype


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,build_existing_model.ahs_region,build_existing_model.applicable,...,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,sqft,gas_eui_thermpersqft,elec_eui_kwhpersqft,site_eui_kbtupersqft,vintage_ee,stories
0,1,4,2021-05-24 09:32:54,2021-05-24 09:36:44,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,336.390504,319.082970,2514.0,395.017450,1202.0,0.445445,4.095515,58.508320,3: post-1978,2: 2+ stories
1,2,43,2021-05-24 09:30:30,2021-05-24 09:33:46,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,53.244335,67.353566,2155.0,74.409253,866.0,0.873858,7.004861,111.266492,2: 1942-1978,1: <2 stories
2,3,6,2021-05-24 09:46:18,2021-05-24 09:50:02,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,141.767218,90.609550,4744.0,170.939906,1202.0,0.447239,3.904510,58.036003,3: post-1978,2: 2+ stories
3,4,85,2021-05-24 10:38:05,2021-05-24 10:41:27,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,247.485970,335.623777,544.0,421.112130,1202.0,0.000000,11.554596,39.425885,3: post-1978,1: <2 stories
4,5,25,2021-05-24 08:54:48,2021-05-24 08:58:53,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,257.171182,278.261169,6378.0,376.538900,1675.0,0.288625,5.129941,46.359645,3: post-1978,2: 2+ stories
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2460,51,2021-05-24 09:06:45,2021-05-24 09:08:02,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,6.247695,8.579762,8561.0,9.477889,1220.0,1.287834,6.335857,150.371562,1: pre-1942,1: <2 stories
2460,2461,51,2021-05-24 09:48:06,2021-05-24 09:51:51,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,114.457268,84.159643,3955.0,132.491715,1675.0,0.515991,3.814951,64.603972,2: 1942-1978,2: 2+ stories
2461,2462,21,2021-05-24 09:06:30,2021-05-24 09:10:23,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,47.371967,25.414189,4456.0,52.041861,1202.0,0.876230,4.198847,101.929126,3: post-1978,2: 2+ stories
2462,2463,32,2021-05-24 09:49:23,2021-05-24 09:53:21,Success,None,None,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,103.564205,79.489322,5005.0,117.415387,1675.0,0.678793,4.000522,81.513476,3: post-1978,2: 2+ stories


In [11]:
### (2) get mean consumption
Metric_map2 = {'mean gas': 'simulation_output_report.total_site_natural_gas_therm',
           'mean elec': 'simulation_output_report.total_site_electricity_kwh'} # metric: res_var

for i, row in groups.iterrows():
    res_group_i = get_res_by_prototype(res, res, row) # <----
    
    for metric, res_var in Metric_map2.items():
        groups.loc[i, metric] = res_group_i[res_var].mean()

groups['Gas Diff-mean(%)'] = ((groups['Non-normalized gas usage'] - groups['mean gas']) / groups['Non-normalized gas usage']) * 100
groups['Elec Diff-mean(%)'] = ((groups['Non-normalized elec. usage'] - groups['mean elec']) / groups['Non-normalized elec. usage']) * 100
print('>> (2) mean consumption computed')


>> (2) mean consumption computed


In [12]:
### (3) get median & mean sqft & eui
Metric_map1 = {'median sqft': 'sqft',
               'median gas eui': 'gas_eui_thermpersqft', 
               'median elec eui': 'elec_eui_kwhpersqft'}
Metric_map2 = {'mean sqft': 'sqft', 
               'mean gas eui': 'gas_eui_thermpersqft', 
               'mean elec eui': 'elec_eui_kwhpersqft'}

for i, row in groups.iterrows():
    res_group_i = get_res_by_prototype(res, res, row) # <----
    
    ### (3.1) get median values
    for metric, res_var in Metric_map1.items():
        groups.loc[i, metric] = res_group_i[res_var].median()

    ### (3.2) get mean values
    for metric, res_var in Metric_map2.items():
        groups.loc[i, metric] = \
            res_group_i[res_var].mean()

print('>> (3) median & mean sqft & eui computed:');

groups.set_index(['HousingGroupNo','HousingGroupName'])[['median sqft', 'mean sqft',
                                                         'median gas','mean gas',
                                                         'median gas eui', 'mean gas eui', 
                                                         'median elec','mean elec', 
                                                         'median elec eui','mean elec eui']]


>> (3) median & mean sqft & eui computed:


,,median sqft,mean sqft,median gas,mean gas,median gas eui,mean gas eui,median elec,mean elec,median elec eui,mean elec eui
HousingGroupNo,HousingGroupName,,,,,,,,,,
1,Masonry Post-1978 less than 2 stories,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Masonry All Years Split Level,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Masonry Post-1978 2 or more stories,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Masonry 1942-1978 Less than 2 Stories,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Masonry 1942-1978 2 or more stories,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Masonry Pre-1942 less than 2 stories,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Masonry Pre-1942 2 or more stories,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Frame Post-1978 Less than 2 stories,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Frame Post-1978 Split Level,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
### export BASELINE prototype summary
groups.to_csv(os.path.join(result_path, 'baseline', 'baseline_prototype_results.csv'), index=True)
print(f'BASELINE prototype summary saved to:\n  {os.path.join(result_path, "baseline")}')


BASELINE prototype summary saved to:
  /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_03_sfamh/baseline


## 2A. PACKAGE post processing ###
Three packages were ran as partitioned files. See "Cookcnty_packages_02.yml" for info
#### (1) Packages WITHOUT partitions (N=1, pkg 06)

In [14]:
def check_and_correct_for_missing_buildings(Pi, res, upgrade_no):
    delta = len(res)-len(Pi)
    if delta>0:
        print(f'   - upgrade_{upgrade_no:02d} is missing {delta} buildings compared to baseline')
    # always reindex
    Pi = Pi.set_index('building_id').reindex(res['building_id']).reset_index()
        
    return Pi

'func "check_and_correct_for_missing_buildings" loaded'

'func "check_and_correct_for_missing_buildings" loaded'

#### (2) Packages WITH single-level partitions (N=10, pkg 07-16)

In [15]:
# 2 partitiions over 'Attic Insulation'

def combine_df_from_first_level_partitions(pkg_no, upgrade_list, partition_para, options_list_for_first_upgrades, 
                                           MSHP_option=None, nullify_total_ng=False, save_as_csv=True, 
                                           recreate_from_scratch=True):
    global res
    """
    ARGS:
        pkg_no (int): package number to assign to combined df
        upgrade_list (list): list of ResStock upgrades to combine
        partition_para (str): para to filter the upgrades by and combine
        options_list_for_first_upgrades (list of list): list of para options to filter the first n-1 upgrades by
        MSHP_option (int): option number in package to correct, default to None
        nullify_total_ng (bool): default to False, set total NG use to 0, for electrification package only
        save_as_csv (bool): default to True
        recreate_from_scratch (bool): default to True
    
    RETURN:
        P1: combined df
    """
    file = f'package{pkg_no:02d}.csv'
    filename = os.path.join(combined_res_csv_path, file)
    print(f'* {file}, from upgrades: {upgrade_list}')
    
    N = len(upgrade_list)
    partition_para = f'build_existing_model.{partition_para}'
    
    # check setting:
    if N-1 != len(options_list_for_first_upgrades):
        raise ValueError(f'The size of "options_list_for_first_upgrades" = {len(options_list_for_first_upgrades)}, '+
                         f'is not 1 less than the size of "upgrade_list" = {N}')

    if not os.path.exists(filename) or recreate_from_scratch:
        print(f'building from resstock results... partitioned by:\n  "{partition_para}"')

        ## (1) building ids for each partitions based on partition_para and para_options
        bldg_list = options_list = []; msg = ' '; n_bldgs = 0
        for n in range(N-1):
            bldgs_for_part_n = res[res[partition_para].isin(options_list_for_first_upgrades[n])]['building_id']
            bldg_list.append(bldgs_for_part_n)
            
            options_for_part_n = options_list_for_first_upgrades[n]
            options_list = options_list + options_for_part_n
            
            n_bldgs += len(bldgs_for_part_n)
            msg += f' part {n+1}: {len(bldgs_for_part_n)},'
        
        # for last partition:
        bldgs_for_part_n = res[~res[partition_para].isin(options_list)]['building_id']
        bldg_list.append(bldgs_for_part_n)
        
        n_bldgs += len(bldgs_for_part_n)
        msg += f' part {N}: {len(bldgs_for_part_n)}, total: {n_bldgs}'
        print(msg)

        ## (2) combine partitions and update upgrade name
        P1 = []
        for up, Bi in zip(upgrade_list, bldg_list):
            p = load_upgrade(up, result_path)
            p = check_and_correct_for_missing_buildings(p, res, up)
            P1.append(p[p['building_id'].isin(Bi)])

        P1 = pd.concat(P1, axis=0).sort_index()

        P1['apply_upgrade.upgrade_part'] = P1['apply_upgrade.upgrade_name'] # new col to show partition #
        P1['apply_upgrade.upgrade_name'] = P1['apply_upgrade.upgrade_name'].apply(
            lambda x: ' '.join(str(x).split(' ')[:-3])) # update name
        
        ## (3) correct MSHP costs
        if not MSHP_option == None:
            print('\nrecalculating MSHP cost...')
            MSHP_cost = f'simulation_output_report.option_{MSHP_option:02d}_cost_usd'
            
            # rename orig upgrade_cost col
            if not 'simulation_output_report.upgrade_cost_usd_orig' in P1.columns:
                P1['simulation_output_report.upgrade_cost_usd_orig'] = P1['simulation_output_report.upgrade_cost_usd']

            # copy upgrade_cost col
            P1['simulation_output_report.upgrade_cost_usd'] = P1['simulation_output_report.upgrade_cost_usd_orig']

            # remove old MSHP cost
            P1['simulation_output_report.upgrade_cost_usd'] -= P1[MSHP_cost]

            # recalc upgrade costs for MSHP
            idx = (P1['simulation_output_report.applicable']==True) & (res['build_existing_model.hvac_has_ducts']=='No')
            P1.loc[idx, MSHP_cost] = \
                710 + (95+1800/12)*P1.loc[idx, 'simulation_output_report.size_heating_system_kbtu_h']

            # calculate backup heating system size and cost
            supp_heat_cost = 38 # <--- 38(avg) 28-47 [$/kBtu_h]
            P1['simulation_output_report.size_heating_supp_system_kbtu_h_mshp'] = np.nan
            P1.loc[idx, 'simulation_output_report.size_heating_supp_system_kbtu_h_mshp'] = \
                P1.loc[idx, 'simulation_output_report.hvac_heating_supp_capacity_w']*3.412142/1000

            P1['simulation_output_report.upgrade_cost_usd_supp_heat_mshp'] = np.nan
            P1.loc[idx, 'simulation_output_report.upgrade_cost_usd_supp_heat_mshp'] = \
                supp_heat_cost * P1.loc[idx, 'simulation_output_report.size_heating_supp_system_kbtu_h_mshp']

            # add new MSHP and backup heat cost to 'upgrade_cost_usd'
            P1.loc[idx, 'simulation_output_report.upgrade_cost_usd'] += \
                P1.loc[idx, MSHP_cost]

            P1.loc[idx, 'simulation_output_report.upgrade_cost_usd'] += \
                P1.loc[idx, 'simulation_output_report.upgrade_cost_usd_supp_heat_mshp']

            # show
            print(f'>> package={pkg_no:02d} modified cols:')
            display(P1.loc[idx, ['apply_upgrade.upgrade_name',
                                'simulation_output_report.upgrade_cost_usd_orig',
                                'simulation_output_report.upgrade_cost_usd', # updated
                                'simulation_output_report.upgrade_cost_usd_supp_heat_mshp', # new
                                'simulation_output_report.size_heating_supp_system_kbtu_h_mshp', # new
                               ]])
            
        ## (4) manually set total NG therm to 0 if pkg is electrification
        if nullify_total_ng:
            P1.loc[P1['simulation_output_report.applicable']==True,
                   'simulation_output_report.total_site_natural_gas_therm'] = 0

        ## (5) save
        if save_as_csv:
            print(f'\n>> combined file saved to: {combined_res_csv_path}')
            P1.to_csv(filename, index=False)
            
    else:
        P1 = pd.read_csv(filename)

#     display(P1)
    return P1

print('func "combine_df_from_two_partitions"')


func "combine_df_from_two_partitions"


In [16]:
pkg_no = 11
upgrade_list = [1, 2]
partition_para = 'infiltration'
options_list_for_first_upgrades = [
    ['50 ACH50','40 ACH50','30 ACH50','25 ACH50','20 ACH50','15 ACH50','10 ACH50','8 ACH50'],
]
MSHP_option = None

combine_df_from_first_level_partitions(pkg_no, upgrade_list, partition_para, options_list_for_first_upgrades, 
                                       MSHP_option, nullify_total_ng=False, save_as_csv=True, 
                                       recreate_from_scratch=True)


* package11.csv, from upgrades: [1, 2]
building from resstock results... partitioned by:
  "build_existing_model.infiltration"
  part 1: 2134, part 2: 330, total: 2464

>> combined file saved to: /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_03_sfamh/processed results/raw combined csvs


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,89,2021-05-24 10:35:42,2021-05-24 10:43:25,Success,True,Comprehensive - Forced Air + WH + Shell,,True,600.0,...,34.829336,34.750402,29.625017,16.909091,18.272727,305.500761,282.368157,2514.0,347.987028,Comprehensive - Forced Air + WH + Shell - Part 1
1,2,73,2021-05-24 10:07:07,2021-05-24 10:07:51,Invalid,True,Comprehensive - Forced Air + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Forced Air + WH + Shell - Part 1
2,3,47,2021-05-24 09:42:16,2021-05-24 09:49:16,Success,True,Comprehensive - Forced Air + WH + Shell,,True,400.0,...,32.136019,25.596135,24.221493,16.000000,17.181818,130.836291,86.089186,4744.0,161.569174,Comprehensive - Forced Air + WH + Shell - Part 1
3,4,18,2021-05-24 09:25:41,2021-05-24 09:26:22,Invalid,True,Comprehensive - Forced Air + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Forced Air + WH + Shell - Part 2
4,5,1,2021-05-24 09:15:43,2021-05-24 09:22:38,Success,True,Comprehensive - Forced Air + WH + Shell,,True,160.0,...,19.499049,16.253957,15.620635,13.272727,13.909091,253.014836,273.344194,6377.0,365.987007,Comprehensive - Forced Air + WH + Shell - Part 2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2460,83,2021-05-24 09:47:50,2021-05-24 09:48:31,Invalid,True,Comprehensive - Forced Air + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Forced Air + WH + Shell - Part 1
2460,2461,3,2021-05-24 08:53:59,2021-05-24 08:54:46,Invalid,True,Comprehensive - Forced Air + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Forced Air + WH + Shell - Part 1
2461,2462,76,2021-05-24 10:46:18,2021-05-24 10:53:25,Success,True,Comprehensive - Forced Air + WH + Shell,,True,100.0,...,5.441984,6.658999,5.444981,15.000000,16.181818,43.786137,22.582347,4456.0,51.413334,Comprehensive - Forced Air + WH + Shell - Part 1
2462,2463,38,2021-05-24 09:40:05,2021-05-24 09:47:02,Success,True,Comprehensive - Forced Air + WH + Shell,,True,160.0,...,15.045559,12.427675,11.333981,13.545455,12.363636,79.482086,60.004641,5005.0,93.440681,Comprehensive - Forced Air + WH + Shell - Part 1


In [17]:
pkg_no = 13
upgrade_list = [3, 4]
partition_para = 'infiltration'
options_list_for_first_upgrades = [
    ['50 ACH50','40 ACH50','30 ACH50','25 ACH50','20 ACH50','15 ACH50','10 ACH50','8 ACH50'],
]
MSHP_option = None

combine_df_from_first_level_partitions(pkg_no, upgrade_list, partition_para, options_list_for_first_upgrades, 
                                       MSHP_option, nullify_total_ng=False, save_as_csv=True, 
                                       recreate_from_scratch=True)


* package13.csv, from upgrades: [3, 4]
building from resstock results... partitioned by:
  "build_existing_model.infiltration"
  part 1: 2134, part 2: 330, total: 2464

>> combined file saved to: /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_03_sfamh/processed results/raw combined csvs


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,67,2021-05-24 10:03:51,2021-05-24 10:04:37,Invalid,True,Comprehensive - Hydronic + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell - Part 1
1,2,31,2021-05-24 09:31:26,2021-05-24 09:32:09,Invalid,True,Comprehensive - Hydronic + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell - Part 1
2,3,8,2021-05-24 09:20:32,2021-05-24 09:21:14,Invalid,True,Comprehensive - Hydronic + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell - Part 1
3,4,81,2021-05-24 10:16:44,2021-05-24 10:17:24,Invalid,True,Comprehensive - Hydronic + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell - Part 2
4,5,57,2021-05-24 10:28:12,2021-05-24 10:28:54,Invalid,True,Comprehensive - Hydronic + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell - Part 2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2460,75,2021-05-24 09:46:09,2021-05-24 09:50:49,Success,True,Comprehensive - Hydronic + WH + Shell,,True,20.0,...,2.388173,2.398394,2.364189,16.727273,17.000000,5.715720,7.556630,8561.0,8.419346,Comprehensive - Hydronic + WH + Shell - Part 1
2460,2461,60,2021-05-24 10:27:54,2021-05-24 10:35:02,Success,True,Comprehensive - Hydronic + WH + Shell,,True,240.0,...,27.946606,19.320933,19.676340,15.818182,18.818182,110.599339,82.504957,3955.0,125.332664,Comprehensive - Hydronic + WH + Shell - Part 1
2461,2462,48,2021-05-24 09:40:08,2021-05-24 09:40:53,Invalid,True,Comprehensive - Hydronic + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell - Part 1
2462,2463,68,2021-05-24 10:25:16,2021-05-24 10:25:59,Invalid,True,Comprehensive - Hydronic + WH + Shell,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Comprehensive - Hydronic + WH + Shell - Part 1


#### (3) Packages WITH multi-level partitions (N=4, pkg 17-20)

In [18]:
def combine_df_from_two_level_partitions(pkg_no, upgrade_list, level1_list, level2_list, MSHP_option=None, 
                                         fix_upgrade_name=False, nullify_total_ng=False, save_as_csv=True, 
                                         recreate_from_scratch=True):
    global res
    """
    ARGS:
        pkg_no (int): package number to assign to combined df
        upgrade_list (list): list of ResStock upgrades to combine
        level1_list (list of dict): list of dictionaries defining the level 1 partition key and options
        level2_list (list of dict): list of dictionaries defining the level 2 partition key and options
        MSHP_option (int): option number in package to correct, default to None
        fix_upgrade_name (bool): default to True, for pkg 21-24 only
        nullify_total_ng (bool): default to False, set total NG use to 0, for electrification package only
        save_as_csv (bool): default to True
        recreate_from_scratch (bool): default to True
    
    RETURN:
        P1: combined df
    """
    
    file = f'package{pkg_no:02d}.csv'
    filename = os.path.join(combined_res_csv_path, file)
    print(f'* {file}, from upgrades: {upgrade_list}')
    
    N_upgrades = len(upgrade_list)
    N_partitions = len(level1_list)*len(level2_list)
    if N_upgrades != N_partitions:
        raise ValueError(f'The size of "upgrade_list" = {N_upgrades} does not match the number of enumeration from '+
                        f'"level1_list" and "level2_list" = {N_partitions}')
    
    if not os.path.exists(filename) or recreate_from_scratch:
        
        print(f'building from resstock run results, {N_partitions} partitions...\n')

        ## building ids for each partitions
        bldg_list = []; CBi_len = 0

        ##### level 1
        P4C1 = level1_list # <----
        for i , Ci in enumerate(P4C1,1):
            CBi = set()
            for key, lst in Ci.items():
                Bi = res[res[f'build_existing_model.{key}'].replace({np.nan:'None'}).isin(lst)]['building_id']
                CBi = CBi.union(set(list(Bi)))
                print(f'- 1.{i} {key}, {len(Bi)} / {len(res)}')
            print(f'- 1.{i} total, {len(CBi)}')
            res1 = res[res[f'building_id'].isin(CBi)]

            ##### level 2
            P4C2 = level2_list # <----
            CBi_2all = set(); CBi_2len = 0
            for k, Ci in enumerate(P4C2,1):
                if k == len(P4C2):
                    CBi = set(list(res1['building_id'])).difference(CBi_2all)
                    print(f'    + 3.{k} total, {len(CBi)}')
                else:
                    CBi = set()
                    for key, lst in Ci.items():
                        Bi = res1[res1[f'build_existing_model.{key}'].isin(lst)]['building_id']
                        CBi = CBi.union(set(list(Bi)))
                        print(f'    + 3.{k} {key}, {len(Bi)} / {len(res1)}')
                CBi_2all = CBi_2all.union(CBi)
                bldg_list.append(CBi)

                CBi_2len += len(CBi)
                CBi_len += len(CBi)
                print(f'appending total {len(CBi)}, (1.{i})(2.{k})_culm = {CBi_2len}, overall_culm = {CBi_len}\n')

        ## (2) combine partitions, update upgrade name
        P4 = []
        for up, Bi in zip(upgrade_list, bldg_list):
            p = load_upgrade(up, result_path)
            p = check_and_correct_for_missing_buildings(p, res, up)
            P4.append(p[p['building_id'].isin(Bi)])

        P4 = pd.concat(P4, axis=0).sort_index()

        P4['apply_upgrade.upgrade_part'] = P4['apply_upgrade.upgrade_name'] # new col to show partition #
        P4['apply_upgrade.upgrade_name'] = P4['apply_upgrade.upgrade_name'].apply(
            lambda x: ' '.join(str(x).split(' ')[:-3])) # update name
        
        ## (3) correct MSHP costs
        if not MSHP_option == None:
            print('\nrecalculating MSHP cost...')
            MSHP_cost = f'simulation_output_report.option_{MSHP_option:02d}_cost_usd'
            
            # rename orig upgrade_cost col
            if not 'simulation_output_report.upgrade_cost_usd_orig' in P4.columns:
                P4['simulation_output_report.upgrade_cost_usd_orig'] = P4['simulation_output_report.upgrade_cost_usd']

            # copy upgrade_cost col
            P4['simulation_output_report.upgrade_cost_usd'] = P4['simulation_output_report.upgrade_cost_usd_orig']

            # remove old MSHP cost
            P4['simulation_output_report.upgrade_cost_usd'] -= P4[MSHP_cost]

            # recalc upgrade costs for MSHP
            idx = (P4['simulation_output_report.applicable']==True) & (res['build_existing_model.hvac_has_ducts']=='No')
            P4.loc[idx, MSHP_cost] = \
                710 + (95+1800/12)*P4.loc[idx, 'simulation_output_report.size_heating_system_kbtu_h']

            # calculate backup heating system size and cost
            supp_heat_cost = 38 # <--- 38(avg) 28-47 [$/kBtu_h]
            P4['simulation_output_report.size_heating_supp_system_kbtu_h_mshp'] = np.nan
            P4.loc[idx, 'simulation_output_report.size_heating_supp_system_kbtu_h_mshp'] = \
                P4.loc[idx, 'simulation_output_report.hvac_heating_supp_capacity_w']*3.412142/1000

            P4['simulation_output_report.upgrade_cost_usd_supp_heat_mshp'] = np.nan
            P4.loc[idx, 'simulation_output_report.upgrade_cost_usd_supp_heat_mshp'] = \
                supp_heat_cost * P4.loc[idx, 'simulation_output_report.size_heating_supp_system_kbtu_h_mshp']

            # add new MSHP and backup heat cost to 'upgrade_cost_usd'
            P4.loc[idx, 'simulation_output_report.upgrade_cost_usd'] += \
                P4.loc[idx, MSHP_cost]

            P4.loc[idx, 'simulation_output_report.upgrade_cost_usd'] += \
                P4.loc[idx, 'simulation_output_report.upgrade_cost_usd_supp_heat_mshp']

            # show
            print(f'>> package={pkg_no:02d} modified cols:')
            display(P4.loc[idx, ['apply_upgrade.upgrade_name',
                                'simulation_output_report.upgrade_cost_usd_orig',
                                'simulation_output_report.upgrade_cost_usd', # updated
                                'simulation_output_report.upgrade_cost_usd_supp_heat_mshp', # new
                                'simulation_output_report.size_heating_supp_system_kbtu_h_mshp', # new
                               ]])
            
        ## (4) fix upgrade name
        try:
            P4[['apply_upgrade.upgrade_name','apply_upgrade.upgrade_part']] = \
                P4[['apply_upgrade.upgrade_name','apply_upgrade.upgrade_part']].apply(
                lambda x: x.str.replace('low-E Window','Low-Gain Window')) # fix upgrade name
        except:
            pass
        
        ## (5) manually set total NG therm to 0 if pkg is electrification
        if nullify_total_ng:
            P4.loc[P4['simulation_output_report.applicable']==True,
                   'simulation_output_report.total_site_natural_gas_therm'] = 0

        ## (6) save
        P4.to_csv(filename, index=False)

    else:              
        P4 = pd.read_csv(filename)
    
    return P4

print('func "combine_df_from_two_level_partitions" loaded')

func "combine_df_from_two_level_partitions" loaded


In [19]:
pkg_no = 17 # <----
upgrade_list = list(range(5, 5+8)) # <----
MSHP_option = None 
fix_upgrade_name = False
    
## (1) get partition keys and options
# para 1 -- 4 parts #######
key = 'infiltration'
PC = [
    {key: ['50 ACH50','40 ACH50','30 ACH50','25 ACH50']},
    {key: ['20 ACH50','15 ACH50','10 ACH50','8 ACH50']},
    {key: ['7 ACH50','6 ACH50','5 ACH50']},
]

options = []
for Ci in PC:
    options = options + list(Ci.values())
options = [item for sublist in options for item in sublist] 
options = list(set(res[f'build_existing_model.{key}'].replace(np.nan, 'None').unique()) - set(options))
PC.append({key: options})
level1_list = PC.copy() # <----

# para 2 -- 2 parts #######
key = 'insulation_unfinished_attic'
PC = [
    {key: ['Uninsulated, Vented', 'Ceiling R-7, Vented', 'Ceiling R-13, Vented']},
]

options = []
for Ci in PC:
    options = options + list(Ci.values())
options = [item for sublist in options for item in sublist] 
options = list(set(res[f'build_existing_model.{key}'].replace(np.nan, 'None').unique()) - set(options))
PC.append({key: options})
level2_list = PC.copy() # <----
    
# (2) create df
combine_df_from_two_level_partitions(pkg_no, upgrade_list, level1_list, level2_list, MSHP_option,
                                     fix_upgrade_name, nullify_total_ng=True, save_as_csv=True, 
                                     recreate_from_scratch=True)


* package17.csv, from upgrades: [5, 6, 7, 8, 9, 10, 11, 12]
building from resstock run results, 8 partitions...

- 1.1 infiltration, 694 / 2464
- 1.1 total, 694
    + 3.1 insulation_unfinished_attic, 204 / 694
appending total 204, (1.1)(2.1)_culm = 204, overall_culm = 204

    + 3.2 total, 490
appending total 490, (1.1)(2.2)_culm = 694, overall_culm = 694

- 1.2 infiltration, 1440 / 2464
- 1.2 total, 1440
    + 3.1 insulation_unfinished_attic, 215 / 1440
appending total 215, (1.2)(2.1)_culm = 215, overall_culm = 909

    + 3.2 total, 1225
appending total 1225, (1.2)(2.2)_culm = 1440, overall_culm = 2134

- 1.3 infiltration, 265 / 2464
- 1.3 total, 265
    + 3.1 insulation_unfinished_attic, 19 / 265
appending total 19, (1.3)(2.1)_culm = 19, overall_culm = 2153

    + 3.2 total, 246
appending total 246, (1.3)(2.2)_culm = 265, overall_culm = 2399

- 1.4 infiltration, 65 / 2464
- 1.4 total, 65
    + 3.1 insulation_unfinished_attic, 2 / 65
appending total 2, (1.4)(2.1)_culm = 2, overall_cul

,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,70,2021-05-24 10:32:20,2021-05-24 10:42:26,Success,True,Electrification - Gas to ASHP,,True,600.0,...,32.772432,83.846925,29.637569,11.636364,12.545455,460.748806,826.988887,895.0,1067.235211,Electrification - Gas to ASHP - Part 4
1,2,23,2021-05-24 09:25:53,2021-05-24 09:26:35,Invalid,True,Electrification - Gas to ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to ASHP - Part 4
2,3,45,2021-05-24 09:08:36,2021-05-24 09:17:19,Success,True,Electrification - Gas to ASHP,,True,400.0,...,27.055234,47.661369,23.727617,16.000000,13.272727,197.595313,595.712850,175.0,872.537122,Electrification - Gas to ASHP - Part 4
3,4,96,2021-05-24 10:20:47,2021-05-24 10:21:28,Invalid,True,Electrification - Gas to ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to ASHP - Part 6
4,5,47,2021-05-24 09:16:06,2021-05-24 09:26:07,Success,True,Electrification - Gas to ASHP,,True,160.0,...,17.317944,28.779185,15.490616,11.818182,13.181818,186.738253,286.220763,8478.0,342.316237,Electrification - Gas to ASHP - Part 8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2460,97,2021-05-24 10:01:27,2021-05-24 10:02:13,Invalid,True,Electrification - Gas to ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to ASHP - Part 2
2460,2461,28,2021-05-24 09:12:37,2021-05-24 09:13:23,Invalid,True,Electrification - Gas to ASHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to ASHP - Part 4
2461,2462,90,2021-05-24 10:01:38,2021-05-24 10:11:32,Success,True,Electrification - Gas to ASHP,,True,100.0,...,5.523835,18.690270,5.702906,14.818182,11.454545,92.205814,265.131970,618.0,312.448365,Electrification - Gas to ASHP - Part 1
2462,2463,2,2021-05-24 09:23:27,2021-05-24 09:32:47,Success,True,Electrification - Gas to ASHP,,True,160.0,...,13.627320,31.843812,11.370922,10.636364,12.272727,180.242835,474.259483,871.0,614.316536,Electrification - Gas to ASHP - Part 1


In [20]:
pkg_no = 18 # <----
upgrade_list = list(range(13, 13+12)) # <----
MSHP_option = None #5 # <----
fix_upgrade_name = False

## (1) get partition keys and options
# para 1 -- 4 parts  #######
key = 'infiltration'
PC = [
    {key: ['50 ACH50','40 ACH50','30 ACH50','25 ACH50']},
    {key: ['20 ACH50','15 ACH50','10 ACH50','8 ACH50']},
    {key: ['7 ACH50','6 ACH50','5 ACH50']},
]

options = []
for Ci in PC:
    options = options + list(Ci.values())
options = [item for sublist in options for item in sublist] 
options = list(set(res[f'build_existing_model.{key}'].replace(np.nan, 'None').unique()) - set(options))
PC.append({key: options})
level1_list = PC.copy() # <----

# para 2 -- 3 parts #######
key = 'insulation_unfinished_attic'
PC = [
    {key: ['Uninsulated, Vented', 'Ceiling R-7, Vented']},
    {key: ['Ceiling R-13, Vented', 'Ceiling R-19, Vented']},
]

options = []
for Ci in PC:
    options = options + list(Ci.values())
options = [item for sublist in options for item in sublist] 
options = list(set(res[f'build_existing_model.{key}'].replace(np.nan, 'None').unique()) - set(options))
PC.append({key: options})
level2_list = PC.copy() # <----
    
# (2) create df
combine_df_from_two_level_partitions(pkg_no, upgrade_list, level1_list, level2_list, MSHP_option,
                                     fix_upgrade_name, nullify_total_ng=True, save_as_csv=True, 
                                     recreate_from_scratch=True)


* package18.csv, from upgrades: [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
building from resstock run results, 12 partitions...

- 1.1 infiltration, 694 / 2464
- 1.1 total, 694
    + 3.1 insulation_unfinished_attic, 128 / 694
appending total 128, (1.1)(2.1)_culm = 128, overall_culm = 128

    + 3.2 insulation_unfinished_attic, 228 / 694
appending total 228, (1.1)(2.2)_culm = 356, overall_culm = 356

    + 3.3 total, 338
appending total 338, (1.1)(2.3)_culm = 694, overall_culm = 694

- 1.2 infiltration, 1440 / 2464
- 1.2 total, 1440
    + 3.1 insulation_unfinished_attic, 85 / 1440
appending total 85, (1.2)(2.1)_culm = 85, overall_culm = 779

    + 3.2 insulation_unfinished_attic, 454 / 1440
appending total 454, (1.2)(2.2)_culm = 539, overall_culm = 1233

    + 3.3 total, 901
appending total 901, (1.2)(2.3)_culm = 1440, overall_culm = 2134

- 1.3 infiltration, 265 / 2464
- 1.3 total, 265
    + 3.1 insulation_unfinished_attic, 8 / 265
appending total 8, (1.3)(2.1)_culm = 8, overall_

,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,apply_upgrade.upgrade_part
0,1,56,2021-05-24 10:03:41,2021-05-24 10:04:23,Invalid,True,Electrification - Gas to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to MSHP - Part 6
1,2,71,2021-05-24 10:27:23,2021-05-24 10:36:30,Success,True,Electrification - Gas to MSHP,,True,200.0,...,22.447561,21.451160,20.411847,12.090909,12.454545,274.482353,396.935244,189.0,475.114156,Electrification - Gas to MSHP - Part 5
2,3,21,2021-05-24 09:43:37,2021-05-24 09:44:20,Invalid,True,Electrification - Gas to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to MSHP - Part 6
3,4,47,2021-05-24 09:41:30,2021-05-24 09:42:13,Invalid,True,Electrification - Gas to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to MSHP - Part 9
4,5,11,2021-05-24 08:54:01,2021-05-24 08:54:42,Invalid,True,Electrification - Gas to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to MSHP - Part 12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2460,11,2021-05-24 09:21:55,2021-05-24 09:27:49,Success,True,Electrification - Gas to MSHP,,True,20.0,...,2.889245,5.635745,2.508157,15.545455,11.818182,24.052483,45.629603,631.0,53.363989,Electrification - Gas to MSHP - Part 3
2460,2461,9,2021-05-24 09:45:35,2021-05-24 09:54:58,Success,True,Electrification - Gas to MSHP,,True,240.0,...,21.672052,40.384820,19.558651,11.363636,14.909091,182.798545,419.584369,886.0,505.468601,Electrification - Gas to MSHP - Part 5
2461,2462,46,2021-05-24 09:48:03,2021-05-24 09:48:47,Invalid,True,Electrification - Gas to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to MSHP - Part 2
2462,2463,99,2021-05-24 10:36:40,2021-05-24 10:37:25,Invalid,True,Electrification - Gas to MSHP,,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Electrification - Gas to MSHP - Part 2


## 2B. PACKAGE summary ###

In [21]:
### upgrade result processing funcs
def assign_utility_rates_to_upgrade(upkg_no, p, ref, HVAC_upgrades_rate_change, for_packages=False):
    
    res = ref.set_index('building_id')[['gas_rate', 'gas_fixed', 'gas_CO2_rate',
                                        'elec_rate', 'elec_fixed', 'elec_CO2_rate'
                                       ]]
    ## assign rates
    p['gas_rate'] = p['building_id'].map(res['gas_rate'])
    p['gas_fixed'] = p['building_id'].map(res['gas_fixed'])
    p['gas_CO2_rate'] = p['building_id'].map(res['gas_CO2_rate'])
    p['elec_rate'] = p['building_id'].map(res['elec_rate'])
    p['elec_fixed'] = p['building_id'].map(res['elec_fixed'])
    p['elec_CO2_rate'] = p['building_id'].map(res['elec_CO2_rate'])
    
    if not for_packages:
        # ind upgrades
        for n in HVAC_upgrades_rate_change.keys():
            new_rates = HVAC_upgrades_rate_change[n]
            p.loc[p['simulation_output_report.applicable']==True, 'gas_rate'] = new_rates[0]
            p.loc[p['simulation_output_report.applicable']==True, 'gas_fixed'] = new_rates[1]
            p.loc[p['simulation_output_report.applicable']==True, 'elec_rate'] = new_rates[2]
            p.loc[p['simulation_output_report.applicable']==True, 'elec_fixed'] = new_rates[3]
    else:
        if isinstance(HVAC_upgrades_rate_change, dict):
            # packages with a dict input
            if upkg_no in HVAC_upgrades_rate_change.keys():
                for m in HVAC_upgrades_rate_change[upkg_no]:
                    idx = (p[(p['apply_upgrade.applicable']==True) &
                           (~p[f'simulation_output_report.option_{m:02d}_cost_usd'].isnull())
                            ]).index
                    p.loc[idx, 'gas_rate'] = NGH_rate
                    p.loc[idx, 'gas_fixed'] = NGH_fixed
                    p.loc[idx, 'elec_rate'] = EH_rate
                    p.loc[idx, 'elec_fixed'] = EH_fixed
        
        else:
            # packages with a list input
            for m in HVAC_upgrades_rate_change:
                idx = (p[(p['apply_upgrade.applicable']==True) &
                       (~p[f'simulation_output_report.option_{m:02d}_cost_usd'].isnull())
                        ]).index
                p.loc[idx, 'gas_rate'] = NGH_rate
                p.loc[idx, 'gas_fixed'] = NGH_fixed
                p.loc[idx, 'elec_rate'] = EH_rate
                p.loc[idx, 'elec_fixed'] = EH_fixed
                
    # assign 0 rates to building with no energy use  
    p.loc[p['simulation_output_report.total_site_natural_gas_therm'].isin([0, np.nan]), 'gas_fixed'] = 0 
    p.loc[p['simulation_output_report.total_site_natural_gas_therm'].isin([0, np.nan]), 'gas_rate'] = 0 
    p.loc[p['simulation_output_report.total_site_electricity_kwh'].isin([0, np.nan]), 'elec_fixed'] = 0 
    p.loc[p['simulation_output_report.total_site_electricity_kwh'].isin([0, np.nan]), 'elec_rate'] = 0 
    
    for col in ['gas_rate','gas_fixed','gas_CO2_rate','elec_rate','elec_fixed','elec_CO2_rate']:
        p.loc[p['simulation_output_report.applicable']==False, col] = np.nan

    return p

def combine_upgrade_cost_and_lifetime(p):
    
    ## upgrade costs (sum)
    p['upgrade_cost'] = p['simulation_output_report.upgrade_cost_usd']
    cost_cols = list(x for x in p.columns if x.endswith('cost_usd'))
    p = p.drop(cost_cols, axis=1)
    
    ## upgrade lifetime (min)
    lt_cols = list(x for x in p.columns if x.endswith('lifetime_yrs'))
    p['upgrade_lifetime'] = p[lt_cols].min(axis=1)
    p = p.drop(lt_cols, axis=1)

    p['upgrade_cost'] = p['upgrade_cost'].replace([0, None,''],np.nan)
    p['upgrade_lifetime'] = p['upgrade_lifetime'].replace([0, None,''],np.nan)
    
    return p

def get_annual_totals(pp, get_col_only=False):
    if get_col_only:
        p = pp.copy()
    else:
        p = pp
        
    p['ann_gas_cost'] = \
        p['simulation_output_report.total_site_natural_gas_therm']*p['gas_rate']+p['gas_fixed']
    p['ann_elec_cost'] = \
        p['simulation_output_report.total_site_electricity_kwh']*p['elec_rate']+p['elec_fixed']
    p['ann_energy_cost'] = \
        p['ann_gas_cost'] + p['ann_elec_cost']
    
    p['ann_metric_ton_co2e_gas'] = \
        p['simulation_output_report.total_site_natural_gas_therm']*p['gas_CO2_rate']
    p['ann_metric_ton_co2e_elec'] = \
        p['simulation_output_report.total_site_electricity_kwh']*p['elec_CO2_rate']
    p['ann_metric_ton_co2e'] = \
        p['ann_metric_ton_co2e_gas'] + p['ann_metric_ton_co2e_elec']
    
    if get_col_only:
        return p[['ann_gas_cost','ann_elec_cost','ann_energy_cost',
                 'ann_metric_ton_co2e_gas','ann_metric_ton_co2e_elec','ann_metric_ton_co2e']]
    else:
        return p

def get_annual_gas_elec_site_energy_saving(pp, res, get_col_only=False):
    """
    p: upgrade df
    ref: reference scenario df
    get_col_only: whether to return the computed col only or the entire upgrade df p
    """
    if get_col_only:
        p = pp.copy()
    else:
        p = pp
    ref = res.set_index('building_id').reindex(p['building_id']).reset_index()
    
    p['ann_therm_gas_saving'] = ref['simulation_output_report.total_site_natural_gas_therm']-\
         p['simulation_output_report.total_site_natural_gas_therm']
    p['ann_kwh_elec_saving'] = ref['simulation_output_report.total_site_electricity_kwh']-\
         p['simulation_output_report.total_site_electricity_kwh']
    p['ann_mbtu_site_energy_saving'] = ref['simulation_output_report.total_site_energy_mbtu']-\
        p['simulation_output_report.total_site_energy_mbtu']
    
    if get_col_only:
        return p[['ann_therm_gas_saving','ann_kwh_elec_saving','ann_mbtu_site_energy_saving']]
    else:
        return p

def get_annual_energy_cost_saving(pp, res, get_col_only=False):
    """
    p: upgrade df
    ref: reference scenario df
    get_col_only: whether to return the computed col only or the entire upgrade df p
    """ 
    if get_col_only:
        p = pp.copy()
    else:
        p = pp
    ref = res.set_index('building_id').reindex(p['building_id']).reset_index()
    
    p['ann_gas_cost_saving'] = \
        ref['simulation_output_report.total_site_natural_gas_therm']*ref['gas_rate']+ref['gas_fixed'] - \
        (p['simulation_output_report.total_site_natural_gas_therm']*p['gas_rate']+p['gas_fixed'])
        
    p['ann_elec_cost_saving'] = \
        ref['simulation_output_report.total_site_electricity_kwh']*ref['elec_rate']+ref['elec_fixed'] - \
        (p['simulation_output_report.total_site_electricity_kwh']*p['elec_rate']+p['elec_fixed'])
    
    p['ann_energy_cost_saving'] = p['ann_gas_cost_saving']+p['ann_elec_cost_saving'] #p[['ann_gas_cost_saving','ann_elec_cost_saving']].sum(axis=1)
    
    if get_col_only:
        return p['ann_energy_cost_saving']
    else:
        return p

def get_annual_metric_ton_co2e_saving(pp, res, get_col_only=False):
    """
    p: upgrade df
    ref: reference scenario df
    get_col_only: whether to return the computed col only or the entire upgrade df p
    """
    if get_col_only:
        p = pp.copy()
    else:
        p = pp
    ref = res.set_index('building_id').reindex(p['building_id']).reset_index()
    
    p['ann_metric_ton_co2e_saving_gas'] = \
        (ref['simulation_output_report.total_site_natural_gas_therm']-\
         p['simulation_output_report.total_site_natural_gas_therm'])*p['gas_CO2_rate']
    
    p['ann_metric_ton_co2e_saving_elec'] = \
        (ref['simulation_output_report.total_site_electricity_kwh']-\
         p['simulation_output_report.total_site_electricity_kwh'])*p['elec_CO2_rate']
    
    p['ann_metric_ton_co2e_saving'] = p['ann_metric_ton_co2e_saving_gas'] + p['ann_metric_ton_co2e_saving_elec']
    
    if get_col_only:
        return p['ann_metric_ton_co2e_saving']
    else:
        return p

def print_metrics_report(p, has_comparative_payback=True):
    ### (1) check for simple_payback > 100 yr or if < 0 yr
    if len(p[~p['simple_payback'].isnull()]):
        print(p['simple_payback'].agg(['min','median','max']))
    spb_100 = p[p['simple_payback']>100]
    if len(spb_100)>0:
        print(f'    *PAYBACK1 - too large* upgrade={n} has {len(spb_100)} simple_payback>100 ' +\
              f'(including {len(p[p["simple_payback"]==np.inf])} INF)')
    spb_neg = p[p['simple_payback']<0]
    if len(spb_neg)>0:
        print(f'    *PAYBACK1 - negative*  upgrade={n} has {len(spb_neg)} simple_payback<0 ' +\
              '(due to negative energy cost saving)')

    ### (2) check for comparative_payback > 100 yr or if < 0 yr
    if has_comparative_payback:
        if len(p[~p['comparative_payback'].isnull()]):
            print(p['comparative_payback'].agg(['min','median','max']))
        spb_100 = p[p['comparative_payback']>100]
        if len(spb_100)>0:
            print(f'    *PAYBACK2 - too large* upgrade={n} has {len(spb_100)} comparative_payback>100 ' +\
                  f'(including {len(p[p["comparative_payback"]==np.inf])} INF)')
        spb_neg = p[p['comparative_payback']<0]
        if len(spb_neg)>0:
            print(f'    *PAYBACK2 - negative*  upgrade={n} has {len(spb_neg)} comparative_payback<0 ' +\
                  '(due to negative energy cost saving)')

    ### (3) check for eui==inf
    for eui in ['pct_delta_gas_eui','pct_delta_elec_eui','pct_delta_site_eui']:
        eui_inf = p[p[eui]==np.inf]
        if len(eui_inf)>0:
            print(f'       *EUI - inf* upgrade={n} has {len(eui_inf)} {eui}=INF ' +\
                  '(due to fuel introduction from upgrade)')

    ### (4) check for neg carbon savings
    ces_neg = p[p['ann_metric_ton_co2e_saving']<0]
    if len(ces_neg)>0:
        print(f'    *CARBON - negative*  upgrade={n} has {len(ces_neg)} carbon saving<0 ')
        
print('funcs loaded')

funcs loaded


In [22]:
### set utility rates ###
NG_rate_multiplier = 1 # <-----

if NG_rate_multiplier > 1:
    fn_ext = f'_{NG_rate_multiplier}x_gas_prices' # file name extension to add to relevant results
else:
    fn_ext = ''

### utility rates ###########################################
# ref (EIA): 
# avg ComEd res elec rate 2019: $ 0.1330 /kWh
# weighted avg IL gas rate 2019: $ 0.77183 /therm

## electricity ##
# annual fixed rates = monthly x 12
EH_fixed = 15.70 * 12 # annual
NEH_fixed = 14.28 *12 # annual
# avg of summer rates (J,J,A,S) and non-summer rates
EH_rate = (0.10273*4+0.10952*8)/12 # 0.08019, $/kWh, electric rate for electric heating customers
NEH_rate = (0.12168*4+0.12847*8)/12 # 0.09889, $/kWh, electric rate for non-electric heating customers
# marginal carbon emission factor:
elec_CO2_rate = 0.000834702 # metric tons of CO2e/kWh (0.2446 tons/mbtu)

## gas ##
# annual fixed rates = monthly x 12
GH_fixed = 45.32 * 12
NGH_fixed = 21.51 * 12
# variable rates
GH_rate = 0.61648 * NG_rate_multiplier # 0.19477, $/therm, gas rate for NG heating customers
NGH_rate = 0.56758 * NG_rate_multiplier # 0.14964, $/therm, gas rate for non-NG heating customers
# marginal carbon emission factor:
gas_CO2_rate = 0.00532181 # metric tons of CO2e/therm (0.0532 tons/mbtu)

### upgrades that will cause utility rate change: ###########################################
HVAC_upgrades_rate_change = {
    17: [4,6,7], # ASHP
    18: [4,6,7,8], # MSHP,
}

# assign rates accordingly
Elec_heating_types = ['Electricity Baseboard','Electricity ASHP','Electricity Electric Furnace',
                      'Electricity Electric Boiler', 'Electricity Electric Wall Furnace']
NG_heating_types = ['Natural Gas Fuel Wall/Floor Furnace', 'Natural Gas Fuel Furnace',
                    'Natural Gas Fuel Boiler']

res['gas_rate'] = NGH_rate
res['gas_fixed'] = NGH_fixed
res['gas_CO2_rate'] = gas_CO2_rate
res['elec_rate'] = NEH_rate
res['elec_fixed'] = NEH_fixed
res['elec_CO2_rate'] = elec_CO2_rate

res.loc[(res[res['build_existing_model.hvac_heating_type_and_fuel'].isin(NG_heating_types)]).index,
       'gas_rate'] = GH_rate
res.loc[(res[res['build_existing_model.hvac_heating_type_and_fuel'].isin(NG_heating_types)]).index,
       'gas_fixed'] = GH_fixed
res.loc[(res[res['build_existing_model.hvac_heating_type_and_fuel'].isin(Elec_heating_types)]).index,
        'elec_rate'] = EH_rate
res.loc[(res[res['build_existing_model.hvac_heating_type_and_fuel'].isin(Elec_heating_types)]).index,
        'elec_fixed'] = EH_fixed

# assign 0 gas rates to building with no gas use 
res.loc[res['simulation_output_report.total_site_natural_gas_therm'].isin([0, np.nan]), 'gas_fixed'] = 0 
res.loc[res['simulation_output_report.total_site_natural_gas_therm'].isin([0, np.nan]), 'gas_rate'] = 0 
res.loc[res['simulation_output_report.total_site_electricity_kwh'].isin([0, np.nan]), 'elec_fixed'] = 0 
res.loc[res['simulation_output_report.total_site_electricity_kwh'].isin([0, np.nan]), 'elec_rate'] = 0

for col in ['gas_rate','gas_fixed','gas_CO2_rate','elec_rate','elec_fixed','elec_CO2_rate']:
    res.loc[res['completed_status']!='Success', col] = np.nan

print(f'Natural gas rate multiplier: {NG_rate_multiplier}')


Natural gas rate multiplier: 1


### Get baseline results

In [23]:
#### count upgrades
N_upgrades = 4 # <---
export_all_upgrades_to_csv = True # <------

print(f'>>> {iter_path} has {N_upgrades:,} packages')
print(f'Natural gas rate multiplier: {NG_rate_multiplier}')
if export_all_upgrades_to_csv:
    print(f'    Exporting upgrades results to {combined_res_csv_path}\n')        
    res.to_csv(os.path.join(combined_res_csv_path, f'results_baseline{fn_ext}.csv'), index=False)
    
## get summary table
summary_upgrades = []

## initialize dB table
all_proto_upgrades = []

package_list = [11, 13, 17, 18]
for n in package_list:
    p = pd.read_csv(os.path.join(combined_res_csv_path,
                                     f'package{n:02d}.csv'))
    print(f'\nPackage {n}')
    p['build_existing_model.sample_weight'] = 2173432/40000
    
    ### get sim output at unit level
    p = get_per_unit_sim_output_limited(p, res)

    ### assign utility rates
    p = assign_utility_rates_to_upgrade(n, p, res, HVAC_upgrades_rate_change, for_packages=True)
    
    ### collapse upgrade cost and lifetime cols
    p = combine_upgrade_cost_and_lifetime(p)

    ### check if upgrade has 0 successful sims
    if len(p[p['completed_status']=='Success']) == 0:
        print(f' * upgrade={n} has 0 successful simulations')

    ### calculate metrics
    p = add_sqft_eui(p, res)
    EUIi = ['gas_eui_thermpersqft','elec_eui_kwhpersqft','site_eui_kbtupersqft']
    EUIo = ['gas_eui','elec_eui','site_eui']
    for vari, varo in zip(EUIi, EUIo):
        p[f'pct_delta_{varo}'] = ((p[vari]-res[vari])/res[vari]*100)


    # annual energy saving:
    p = get_annual_gas_elec_site_energy_saving(p, res)

    # annual energy cost saving:
    p = get_annual_energy_cost_saving(p, res)
    
    # annual kBtu saved per upgrade cost:
    p['ann_kbtu_saved_per_dollar'] = p['ann_mbtu_site_energy_saving'].divide(
                            p['upgrade_cost'], axis=0)*1000

    # simple payback
    p['simple_payback'] = p['upgrade_cost']/p['ann_energy_cost_saving']

    # annual metric ton carbon emission savings:
    p = get_annual_metric_ton_co2e_saving(p, res)

    ### check for neg/large paybacks, inf eui, neg carbon savings 
    print_metrics_report(p, has_comparative_payback=False)
    
    ### subset to only those that have been applied with the upgrades successfully:
    upgrade_name = p['apply_upgrade.upgrade_name'].replace('',np.nan).dropna(axis=0).unique()[0]
    p = p[p['apply_upgrade.applicable']==True].reset_index(drop=True)
    p['package_no'] = n
    
    ### export upgrade results to csv 
    if export_all_upgrades_to_csv:
        p.to_csv(os.path.join(combined_res_csv_path, f'results_package{n:02d}{fn_ext}.csv'), index=False)
    
    ### add to dB
    all_proto_upgrades.append(p)
    
    for eui in ['pct_delta_gas_eui','pct_delta_elec_eui','pct_delta_site_eui']:
        p[eui] = p[eui].replace([np.inf, -np.inf], np.nan) # for mean calc
        
    ### add to summary table
    summ = pd.DataFrame()
    summ.loc[0,'upgrade_no'] = n
    summ.loc[0,'upgrade_name'] = upgrade_name
    summ.loc[0,'n_applied'] = len(p)
    summ.loc[0,'n_success'] = len(p[p['completed_status']=='Success'])
    summ.loc[0,'n_fail'] = len(p[p['completed_status']=='Fail'])
    summ['pct_success'] = round(summ['n_success']/summ['n_applied']*100,3)

    p = p[p['completed_status']=='Success'].reset_index(drop=True)
    summ['mean_ann_therm_gas_saving'] = round(p['ann_therm_gas_saving'].mean(), 3)
    summ['mean_ann_kwh_elec_saving'] = round(p['ann_kwh_elec_saving'].mean(), 3)
    summ['mean_ann_mbtu_site_energy_saving'] = round(p['ann_mbtu_site_energy_saving'].mean(), 3)
    summ['mean_pct_delta_gas_eui'] = round(p['pct_delta_gas_eui'].mean(), 3)
    summ['mean_pct_delta_elec_eui'] = round(p['pct_delta_elec_eui'].mean(), 3)
    summ['mean_pct_delta_site_eui'] = round(p['pct_delta_site_eui'].mean(), 3)
    summ['mean_upgrade_cost'] = round(p['upgrade_cost'].mean(),2)
    summ['mean_ann_kbtu_saved_per_dollar'] = round(p['ann_kbtu_saved_per_dollar'].mean(),3) # annual kBtu saved per upgrade cost
    summ['mean_ann_energy_cost_saving'] = round(p['ann_energy_cost_saving'].mean(),2)
    summ['mean_ann_metric_ton_co2e_saving'] = round(p['ann_metric_ton_co2e_saving'].mean(),3)

    summ['median_simple_payback'] = round(p['simple_payback'].median(),3)
    
    p['simple_payback'] = p['simple_payback'].replace([np.inf, -np.inf], np.nan) # for mean calc
    summ['pct_pos_simple_payback_actual'] = round(len(p[p['simple_payback']>=0])/len(p)*100, 3)
    summ['mean_pos_simple_payback_actual'] = round(p.loc[p['simple_payback']>=0, 'simple_payback'].mean(),3)

    # filter: min 1 cent energy cost savings **
    min_energy_cost_saving = 0.1 # <----- **
    summ['pct_pos_simple_payback_filtered'] = round(len(
        p[(p['simple_payback']>=0) & (p['ann_energy_cost_saving']>=min_energy_cost_saving)]
    )/len(p)*100, 3)
    summ['mean_pos_simple_payback_filtered'] = round(
        p.loc[(p['simple_payback']>=0) & (p['ann_energy_cost_saving']>=min_energy_cost_saving), 
           'simple_payback'].mean(),3)

    summary_upgrades.append(summ)
    
summary_upgrades = pd.concat(summary_upgrades).reset_index(drop=True)
summary_upgrades


>>> cookcnty_packages_03_sfamh has 4 packages
Natural gas rate multiplier: 1
    Exporting upgrades results to /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_03_sfamh/processed results/raw combined csvs


Package 11
min       -525.073574
median      35.079630
max       2043.539791
Name: simple_payback, dtype: float64
    *PAYBACK1 - too large* upgrade=11 has 106 simple_payback>100 (including 0 INF)
    *PAYBACK1 - negative*  upgrade=11 has 2 simple_payback<0 (due to negative energy cost saving)
    *CARBON - negative*  upgrade=11 has 2 carbon saving<0 

Package 13
min        11.271199
median     45.055789
max       238.651531
Name: simple_payback, dtype: float64
    *PAYBACK1 - too large* upgrade=13 has 8 simple_payback>100 (including 0 INF)

Package 17
min      -9323.023931
median      24.101725
max       5613.574980
Name: simple_payback, dtype: float64
    *PAYBACK1 - too large* upgrade=17 has 67 simple_payback>100 (including 0 INF)
    *PAYBACK1 - negati

,upgrade_no,upgrade_name,n_applied,n_success,n_fail,pct_success,mean_ann_therm_gas_saving,mean_ann_kwh_elec_saving,mean_ann_mbtu_site_energy_saving,mean_pct_delta_gas_eui,...,mean_pct_delta_site_eui,mean_upgrade_cost,mean_ann_kbtu_saved_per_dollar,mean_ann_energy_cost_saving,mean_ann_metric_ton_co2e_saving,median_simple_payback,pct_pos_simple_payback_actual,mean_pos_simple_payback_actual,pct_pos_simple_payback_filtered,mean_pos_simple_payback_filtered
0,11.0,Comprehensive - Forced Air + WH + Shell,2464.0,1947.0,0.0,79.018,258.720,797.022,28.585,-32.859,...,-27.581,8527.46,3.229,259.96,2.042,35.080,99.897,50.037,99.897,50.037
1,13.0,Comprehensive - Hydronic + WH + Shell,2464.0,230.0,0.0,9.334,367.719,674.876,39.066,-38.555,...,-32.629,12097.54,2.875,311.87,2.520,45.056,100.000,51.437,100.000,51.437
2,17.0,Electrification - Gas to ASHP,2464.0,1947.0,0.0,79.018,748.235,-5676.832,55.167,-100.000,...,-54.999,13952.72,3.844,503.28,-0.756,24.102,96.918,41.160,96.918,41.160
3,18.0,Electrification - Gas to MSHP,2464.0,249.0,0.0,10.106,989.451,-4452.363,83.637,-100.000,...,-67.317,13207.83,5.893,725.53,1.549,17.842,100.000,19.568,100.000,19.568


In [24]:
### export UPGRADE summary
summary_upgrades.to_csv(os.path.join(result_path,'processed results',f'upgrades_summary{fn_ext}.csv'), index=False)
print(f'UPGRADE summary table saved to:\n  {os.path.join(result_path, "processed results")}')

UPGRADE summary table saved to:
  /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_packages_03_sfamh/processed results


In [25]:
### optional - export specific upgrade parquet as csv
save_to_csv = False # <-----
upgrade_list = range(1, 2+1) # [8] # <----- list of upgrades to convert to csv

if save_to_csv:
    for n in upgrade_list:
        p = pd.read_parquet(os.path.join(result_path,'upgrades',
                                         f'upgrade={n}/results_up{n:02d}.parquet'))
        p.to_csv(os.path.join(result_path,'upgrades',
                                         f'upgrade={n}/results_up{n:02d}.csv'), index=False)
        print(f'upgrade {n:02d} parquet file exported as csv')


### *Combine results by building prototypes from Elevate Energy into database

In [26]:
# (1) combine baseline results by non-split-level prototypes (N=12)
print(f'Natural gas rate multiplier: {NG_rate_multiplier}')
### add totals metrics:
res = get_annual_totals(res)

res_proto_upgrades = res.copy()

res_proto_upgrades['package_no'] = 0
res_proto_upgrades['apply_upgrade.upgrade_name'] = 'Baseline'

display(res_proto_upgrades)

Natural gas rate multiplier: 1


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,build_existing_model.ahs_region,build_existing_model.applicable,...,elec_rate,elec_fixed,elec_CO2_rate,ann_gas_cost,ann_elec_cost,ann_energy_cost,ann_metric_ton_co2e_gas,ann_metric_ton_co2e_elec,ann_metric_ton_co2e,package_no
0,1,4,2021-05-24 09:32:54,2021-05-24 09:36:44,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.126207,171.36,0.000835,873.918491,792.651337,1666.569829,2.849427,4.109079,6.958506,0
1,2,43,2021-05-24 09:30:30,2021-05-24 09:33:46,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.126207,171.36,0.000835,1010.367788,936.956127,1947.323916,4.027336,5.063478,9.090814,0
2,3,6,2021-05-24 09:46:18,2021-05-24 09:50:02,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.126207,171.36,0.000835,875.248322,763.675729,1638.924051,2.860907,3.917441,6.778348,0
3,4,85,2021-05-24 10:38:05,2021-05-24 10:41:27,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.107257,188.40,0.000835,0.000000,1678.047517,1678.047517,0.000000,11.592862,11.592862,0
4,5,25,2021-05-24 08:54:48,2021-05-24 08:58:53,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.126207,171.36,0.000835,841.874815,1255.809903,2097.684717,2.572808,7.172303,9.745111,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,2460,51,2021-05-24 09:06:45,2021-05-24 09:08:02,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.126207,171.36,0.000835,1512.427330,1146.905410,2659.332740,8.361403,6.452034,14.813437,0
2460,2461,51,2021-05-24 09:48:06,2021-05-24 09:51:51,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.126207,171.36,0.000835,1076.654594,977.826089,2054.480683,4.599562,5.333782,9.933344,0
2461,2462,21,2021-05-24 09:06:30,2021-05-24 09:10:23,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.126207,171.36,0.000835,1193.133930,808.326872,2001.460802,5.605079,4.212753,9.817832,0
2462,2463,32,2021-05-24 09:49:23,2021-05-24 09:53:21,Success,None,Baseline,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.126207,171.36,0.000835,1244.764501,1017.054945,2261.819445,6.050784,5.593233,11.644016,0


In [27]:
# (2) combine upgrade results
print(f'Natural gas rate multiplier: {NG_rate_multiplier}')

all_proto_upgrades = pd.concat(all_proto_upgrades, axis=0).reset_index(drop=True)
all_proto_upgrades


Natural gas rate multiplier: 1


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,ann_mbtu_site_energy_saving,ann_gas_cost_saving,ann_elec_cost_saving,ann_energy_cost_saving,ann_kbtu_saved_per_dollar,simple_payback,ann_metric_ton_co2e_saving_gas,ann_metric_ton_co2e_saving_elec,ann_metric_ton_co2e_saving,package_no
0,1,89,2021-05-24 10:35:42,2021-05-24 10:43:25,Success,True,Comprehensive - Forced Air + WH + Shell,NaN,True,600.0,...,22.910410,123.027026,109.438588,232.465614,2.725089,36.165410,1.062040,0.723802,1.785842,11
1,2,73,2021-05-24 10:07:07,2021-05-24 10:07:51,Invalid,True,Comprehensive - Forced Air + WH + Shell,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11
2,3,47,2021-05-24 09:42:16,2021-05-24 09:49:16,Success,True,Comprehensive - Forced Air + WH + Shell,NaN,True,400.0,...,20.491800,120.941256,32.491353,153.432609,2.886952,46.261838,1.044034,0.214890,1.258925,11
3,4,18,2021-05-24 09:25:41,2021-05-24 09:26:22,Invalid,True,Comprehensive - Forced Air + WH + Shell,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11
4,5,1,2021-05-24 09:15:43,2021-05-24 09:22:38,Success,True,Comprehensive - Forced Air + WH + Shell,NaN,True,160.0,...,12.180788,55.399467,118.231147,173.630614,1.678821,41.787346,0.478240,0.781954,1.260194,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9851,2460,11,2021-05-24 09:21:55,2021-05-24 09:27:49,Success,True,Electrification - Gas to MSHP,NaN,True,20.0,...,118.066859,1512.427330,-1096.842454,415.584876,6.710388,42.337056,8.361403,-9.543270,-1.181867,18
9852,2461,9,2021-05-24 09:45:35,2021-05-24 09:54:58,Success,True,Electrification - Gas to MSHP,NaN,True,240.0,...,69.527174,1076.654594,-426.576042,650.078552,4.957029,21.575820,4.599562,-4.129494,0.470068,18
9853,2462,46,2021-05-24 09:48:03,2021-05-24 09:48:47,Invalid,True,Electrification - Gas to MSHP,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18
9854,2463,99,2021-05-24 10:36:40,2021-05-24 10:37:25,Invalid,True,Electrification - Gas to MSHP,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18


In [28]:
# (3) combine upgrade and baseline

cols = set(res_proto_upgrades.columns).intersection(set(all_proto_upgrades.columns))
all_proto_upgrades = pd.concat([all_proto_upgrades, res_proto_upgrades[cols]], axis=0).reset_index(drop=True)

# (4) save
print('"all_proto_upgrades" df saved')
all_proto_upgrades.to_csv(
    os.path.join(result_path,'processed results',f'upgrades_dB{fn_ext}.csv'), index=False)

display(all_proto_upgrades)


"all_proto_upgrades" df saved


,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,simulation_output_report.applicable,simulation_output_report.door_area_ft_2,...,ann_mbtu_site_energy_saving,ann_gas_cost_saving,ann_elec_cost_saving,ann_energy_cost_saving,ann_kbtu_saved_per_dollar,simple_payback,ann_metric_ton_co2e_saving_gas,ann_metric_ton_co2e_saving_elec,ann_metric_ton_co2e_saving,package_no
0,1,89,2021-05-24 10:35:42,2021-05-24 10:43:25,Success,True,Comprehensive - Forced Air + WH + Shell,NaN,True,600.0,...,22.910410,123.027026,109.438588,232.465614,2.725089,36.165410,1.062040,0.723802,1.785842,11
1,2,73,2021-05-24 10:07:07,2021-05-24 10:07:51,Invalid,True,Comprehensive - Forced Air + WH + Shell,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11
2,3,47,2021-05-24 09:42:16,2021-05-24 09:49:16,Success,True,Comprehensive - Forced Air + WH + Shell,NaN,True,400.0,...,20.491800,120.941256,32.491353,153.432609,2.886952,46.261838,1.044034,0.214890,1.258925,11
3,4,18,2021-05-24 09:25:41,2021-05-24 09:26:22,Invalid,True,Comprehensive - Forced Air + WH + Shell,NaN,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11
4,5,1,2021-05-24 09:15:43,2021-05-24 09:22:38,Success,True,Comprehensive - Forced Air + WH + Shell,NaN,True,160.0,...,12.180788,55.399467,118.231147,173.630614,1.678821,41.787346,0.478240,0.781954,1.260194,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12315,2460,51,2021-05-24 09:06:45,2021-05-24 09:08:02,Success,None,Baseline,,True,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
12316,2461,51,2021-05-24 09:48:06,2021-05-24 09:51:51,Success,None,Baseline,,True,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
12317,2462,21,2021-05-24 09:06:30,2021-05-24 09:10:23,Success,None,Baseline,,True,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
12318,2463,32,2021-05-24 09:49:23,2021-05-24 09:53:21,Success,None,Baseline,,True,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [29]:
## save metadata for dB
if NG_rate_multiplier==1:
    cols2 = sorted((set(res_proto_upgrades.columns)-set(cols)).union(
                   set(['building_id','completed_status','package_no']))
                  )
    res_proto_meta = res_proto_upgrades[cols2]
    res_proto_meta = res_proto_meta[res_proto_meta['completed_status']=='Success'].reset_index(drop=True)
    
    cols_to_drop = [x for x in res_proto_meta.columns if 
                    x.startswith('simulation_output_report.option') or
                    x == 'simulation_output_report.upgrade_cost_usd']
    
    res_proto_meta = res_proto_meta.drop(cols_to_drop, axis=1)

    res_proto_meta.to_csv(
        os.path.join(result_path,'processed results',f'upgrades_dB_meta{fn_ext}.csv'), index=False)

    print('"res_proto_meta" df saved')
    display(res_proto_meta)
    

"res_proto_meta" df saved


,ann_elec_cost,ann_energy_cost,ann_gas_cost,ann_metric_ton_co2e,ann_metric_ton_co2e_elec,ann_metric_ton_co2e_gas,build_existing_model.ahs_region,build_existing_model.applicable,build_existing_model.ashrae_iecc_climate_zone_2004,build_existing_model.bathroom_spot_vent_hour,...,build_existing_model.vintage,build_existing_model.vintage_acs,build_existing_model.water_heater,build_existing_model.window_areas,build_existing_model.windows,building_id,completed_status,package_no,stories,vintage_ee
0,792.651337,1666.569829,873.918491,6.958506,4.109079,2.849427,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour5,...,1990s,1980-99,Gas Standard,F12 B12 L12 R12,2+ Pane,1,Success,0,2: 2+ stories,3: post-1978
1,936.956127,1947.323916,1010.367788,9.090814,5.063478,4.027336,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour3,...,1960s,1960-79,Gas Standard,F12 B12 L12 R12,2+ Pane,2,Success,0,1: <2 stories,2: 1942-1978
2,763.675729,1638.924051,875.248322,6.778348,3.917441,2.860907,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour18,...,2000s,2000-09,Gas Standard,F9 B9 L9 R9,2+ Pane,3,Success,0,2: 2+ stories,3: post-1978
3,1678.047517,1678.047517,0.000000,11.592862,11.592862,0.000000,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour21,...,1990s,1980-99,Electric Standard,F18 B18 L18 R18,2+ Pane,4,Success,0,1: <2 stories,3: post-1978
4,1255.809903,2097.684717,841.874815,9.745111,7.172303,2.572808,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour7,...,2000s,2000-09,Gas Standard,F9 B9 L9 R9,1 Pane,5,Success,0,2: 2+ stories,3: post-1978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,1146.905410,2659.332740,1512.427330,14.813437,6.452034,8.361403,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour21,...,<1940,<1940,Gas Standard,F12 B12 L12 R12,1 Pane,2460,Success,0,1: <2 stories,1: pre-1942
2460,977.826089,2054.480683,1076.654594,9.933344,5.333782,4.599562,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour4,...,1970s,1960-79,Gas Standard,F9 B9 L9 R9,2+ Pane,2461,Success,0,2: 2+ stories,2: 1942-1978
2461,808.326872,2001.460802,1193.133930,9.817832,4.212753,5.605079,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour19,...,1990s,1980-99,Gas Standard,F6 B6 L6 R6,1 Pane,2462,Success,0,2: 2+ stories,3: post-1978
2462,1017.054945,2261.819445,1244.764501,11.644016,5.593233,6.050784,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,5A,Hour5,...,1990s,1980-99,Gas Standard,F9 B9 L9 R9,1 Pane,2463,Success,0,2: 2+ stories,3: post-1978
